In [43]:
from selenium import webdriver
from slugify import slugify
from pathlib import Path
from time import sleep
import json
from pathlib import Path
from selenium.webdriver.common.keys import Keys

In [37]:
def login(b):
    b.get('https://www.newspapers.com/signin/')
    sleep(1)
    b.find_element_by_css_selector('input[name="username"]').send_keys(json.loads(Path('auth.json').read_text())['username'])
    sleep(1)
    b.find_element_by_css_selector('input[name="password"]').send_keys(Keys.ENTER)
    
    
try:
    b
except:
    b = webdriver.Firefox()
    login(b)

In [75]:
from assistance.constants import abbrev_to_us_state, us_state_to_abbrev, urls
from assistance.data import get_clean_network_data
from assistance.utils import slugify

In [76]:
#!/usr/bin/env python
# coding: utf-8

import community as community_louvain
import copy
from colorama import Fore, Style
import networkx as nx
from collections import Counter
import unicodedata
import re
import pandas as pd
import json
import jellyfish
from pathlib import Path
import datetime
from tqdm import tqdm

try:
    from IPython.display import display, HTML, Markdown, clear_output
except ModuleNotFoundError:
    print("No IPython found.")


In [77]:
alternative_spellings = {
    'Billy': [
        'Billie',
        'Billi'
    ],
    'Billie': [
        'Billy',
        'Billi'
    ],
    'Jean': [
        'Gene',
    ],
    'Gene': [
        'Jean',
    ],
    'Bobbie': [
        "Bobby",
        "Bobbi"
    ],
    "Bobby": [
        'Bobbie',
        'Bobbi'
    ],
    'Lengel': [
        'Lengle'
    ],
    'Jackie': [
        'Jacky'
    ],
    'Johnny': [
        'Johnnie',
        'Johnni'
    ],
    'Johnnie': [
        'Johnny',
        'Johnni'
    ],
    'Max': [
        'Maxie'
    ],
    "Rae": [
        'Ray'
    ],
    'Merry': [
        'Murry'
    ],
    'Del': [
        'Delle'
    ],
    'Francis': [
        'Frances'
    ],
    'Frances': [
        'Francis'
    ],
    'Li': [
        'Lee'
    ],
    'Nicki': [
        'Nickie',
        'Nicky'
    ],
    'Lloyd': ['Loyd'],
    'Tangara': ['Tangarra'],
    'Lester': ['Lestra'],
    'Eddie': ['Eddy','Eddi'],
    'Chickie': ['Chicky', 'Chicki'],
    'Gray': ['Grey'],
    'Grey': ['Gray'],
    'Roni': ['Ronni', 'Ronnie', 'Ronny']
}

alternative_names = {
    'Billy "Senorita" Herrera': [
        'Billy Herrera'
    ]
}




def get_alternative_spellings(performer):
    alternatives = [performer]
    name_alternatives = [performer]
    if performer in alternative_names:
        for name in alternative_names[performer]:
            name_alternatives.append(name)
    for name in name_alternatives:
        print(name)
        name = name.replace('"', '')
        words = name.split(' ')
        for word in words:
            if word in alternative_spellings:
                for alt in alternative_spellings[word]:
                    alternatives.append(name.replace(word, alt))
    return list(set(alternatives))



In [78]:
def has_next(b):
    return len(b.find_elements_by_css_selector('.btn.btn-secondary.btn-block')) == 1

def click_next(b):
    if not has_next(b):
        return False
    back_button = b.find_elements_by_css_selector('.btn.btn-secondary.btn-block')
    if back_button:
        back_button = back_button[0].click()
        return True
    return False


def scroll_to_bottom(b):
    start_len = len(get_search_record_elements(b))
    has_more = True
    max_repetitions = 5
    repetition = 0
    while has_more:
        repetition += 1
        if repetition > max_repetitions:
            has_more = False
        for i in range(0,5):
            b.execute_script('window.scrollTo(0,document.body.scrollHeight);')
            sleep(0.5)
            click_next(b)
            sleep(0.5)
            b.execute_script('window.scrollTo(0,-document.body.scrollHeight);')
            sleep(0.5)
            end_len = len(get_search_record_elements(b))
            if not has_next(b):
                repetition += 1

            if start_len <= end_len:
                has_more = False
                repetition = 0
            if has_next(b):
                has_more = True
                


def get_search_record_elements(b):
    return b.find_elements_by_css_selector('[id*="search-record"]')

def get_dict_for_search_record(b):
    paper = record.find_element_by_css_selector('h2').text
    link = record.find_element_by_css_selector('a').get_attribute('href')
    doc_id = link.split('/')[4]
    location = record.find_element_by_css_selector('[title="Location"]').text
    date = record.find_element_by_css_selector('.ml-n1.mb-1.text-dark:not([title="Location"])').text
    date = pd.to_datetime(date)
    return {
        'paper': paper,
        'id': doc_id,
        'location': location,
        'date': date.strftime('%Y-%m-%d')
    }


In [79]:
MATCHES = re.compile('(\d+) Matches')


In [80]:
df = get_clean_network_data(verbose=False, min_date=datetime.datetime(year=1930, month=1, day=1), max_date=datetime.datetime(year=1940, month=12, day=31),)

newspaper_id_per_performer = {performer: list(set([y for y in x.Newspaper_ID if y])) for performer, x in df.groupby('Performer') if list(set([y for y in x.Newspaper_ID if y]))}
newspaper_id_per_performer

{'Adrian Ames': ['416822923',
  '677021057',
  '488668361',
  '135210673',
  '416641918',
  '416940766',
  '417195597'],
 'Adrian Norris': ['40125917'],
 'Al DeMarco': ['617171826',
  '543772371',
  '693861340',
  '617441826',
  '171133088',
  '617442700',
  '298466318',
  '617443111',
  '617441416',
  '617442715'],
 'Al Ray': ['99488871'],
 'Alberta Vaughn': ['511239508',
  '365614795',
  '76214902',
  '362832400',
  '260604709',
  '365619640',
  '362837118',
  '206612962',
  '364902662',
  '41228006',
  '362665149',
  '180331608',
  '76214419',
  '107714322',
  '511239574'],
 'Allen Carl': ['280239587', '280239595'],
 'Andy Cliff': ['576216465', '417151463', '576216447'],
 'Arica Wild': ['576216465', '576217266', '219421893', '417151463'],
 'Art Bernard': ['228732511', '228655028'],
 'Art West': ['616997011',
  '617001816',
  '294151264',
  '616999452',
  '617000848',
  '5136612',
  '616997684',
  '616998178',
  '617000964',
  '649739903',
  '616997324',
  '616997863'],
 'Arthur Budd

In [81]:
df['Year'] = df.apply(lambda row: pd.to_datetime(row.Date).year, axis=1)

In [82]:
MAX = 10000
i = 0

for groups, rows in df.groupby(['State', 'Performer']):
    if i >= MAX:
        continue
    state, performer = groups
    if not state or 'unnamed' in performer.lower():
        continue

    #dates = dict()
    #for year, month, day in [(pd.to_datetime(x).year, pd.to_datetime(x).month, pd.to_datetime(x).day) for x in rows.Date]:
    #    dates_.setdefault(year, {})
    #    dates_[year].setdefault(month, [])
    #    dates_[year][month].append(day)
    #years = list(dates.keys())
    #years.sort()

    performer_orig = performer
    for year in set(pd.to_datetime(x).year for x in rows.Date):
        if year >= 1940:
            continue
        try:
            short = us_state_to_abbrev[state].lower()
        except:
            continue
        datafile = f'performer-searches-by-year/{slugify(performer_orig)}/{slugify(str(year))}-{slugify(state)}.json'
        if not Path(datafile).exists():
            i += 1 # Only performers who haven't been downloaded count
            
            if not Path(datafile).parent.exists():
                Path(datafile).parent.mkdir(parents=True)
            alt_names = get_alternative_spellings(performer)
            records = []
            for performer in alt_names:
                print(performer_orig, 'in', state, year, f'(spelling {performer})')
                querypath = f"%22{performer.replace(' ', '+')}%22&p_province=us-{short}&dr_year={year}-{year}"
                b.get('https://www.apple.com')
                b.get('https://www.newspapers.com/search/#query=' + querypath)

                sleep(3)

                num_results = 0
                try:
                    search_results = b.find_element_by_css_selector('#SearchResults p')
                except:
                    sleep(3)
                    search_results = b.find_element_by_css_selector('#SearchResults p')
                    
                if search_results:
                    try:
                        num_results = MATCHES.search(search_results.text.replace(',', '')).groups()[0]
                        num_results = int(num_results)
                    except:
                        if 'No Matches' in search_results.text:
                            print('---------> Skipping ahead due to to no results')

                print('--->', num_results, 'numbers of results')
                    
                if num_results > 200:
                    print('---------> Skipping ahead due to too many results')
                    
                elif 200 > num_results > 0:
                    scroll_to_bottom(b)

                    for record in get_search_record_elements(b):
                        records.append(get_dict_for_search_record(record))
                elif num_results == 0:
                    print('no results found')
                else:
                    print('What happened here?!')
                    
            article_df = pd.DataFrame.from_dict(records)
            if len(article_df):
                article_df.set_index('id', inplace=True)
                article_df.drop_duplicates(inplace=True)

                Path(datafile).write_text(article_df.to_json())
                print('----> saved.')
            else:
                Path(datafile).write_text(json.dumps({'error': 'no records found'}))
                print('----> error file saved.')
            

In [84]:
for groups, _ in df.groupby(['Performer', 'State', 'Year']):
    performer, state, year = groups
    
    if state == '':
        continue

    if state != 'DC' and state != 'Washington DC':
        state = abbrev_to_us_state[state]
    
    if performer in newspaper_id_per_performer:
        try:
            newspaper_ids_in_dataset = newspaper_id_per_performer[performer]
            datafile = f'/Volumes/GoogleDrive/My Drive/performer-searches-by-year/{slugify(performer)}/{slugify(str(year))}-{slugify(state)}.json'
            json_data = json.loads(Path(datafile).read_text())
            irrelevant_newspaper_ids = [x for x in Path('irrelevant_newspaper_ids.txt').read_text().splitlines() if x]

            num_relevant = len([x for x in json_data.get('paper', []) if not x in newspaper_id])
            if num_relevant > 5 or num_relevant == 0:
                continue
                
            print(performer, state, year, num_relevant)
            for newspaper_id in [x for x in json_data.get('paper', []) if not x in irrelevant_newspaper_ids]:
                if not newspaper_id in newspaper_ids_in_dataset:
                    #print(performer)
                    #print(newspaper_id, 'not in dataset yet', f'{json_data["date"][newspaper_id]}')
                    # b.execute_script(f"window.open('http://www.newspapers.com/image/{newspaper_id}?terms={performer}');")
                    print(f'http://www.newspapers.com/image/{newspaper_id}?terms={performer.replace(" ", "%20")}')
        except FileNotFoundError:
            print(f'file not found: {datafile}')
            continue

Adrian Ames New York 1934 4
Alberta Vaughn Kentucky 1936 3
http://www.newspapers.com/image/502159114?terms=Alberta%20Vaughn
http://www.newspapers.com/image/502157756?terms=Alberta%20Vaughn
http://www.newspapers.com/image/683340772?terms=Alberta%20Vaughn
Alberta Vaughn Michigan 1936 3
Alberta Vaughn Michigan 1937 2
Alberta Vaughn Minnesota 1938 1
Andy Cliff New York 1934 2
Art Bernard Ohio 1937 3
Arthur Budd New York 1930 4
Arthur Budd New York 1931 3
Arthur Budd New York 1932 1
Arthur Budd New York 1933 3
http://www.newspapers.com/image/685971956?terms=Arthur%20Budd
Arthur G. West New Jersey 1930 2
http://www.newspapers.com/image/479525149?terms=Arthur%20G.%20West
http://www.newspapers.com/image/480174414?terms=Arthur%20G.%20West
Arthur G. West New Jersey 1931 2
http://www.newspapers.com/image/489968212?terms=Arthur%20G.%20West
http://www.newspapers.com/image/489752785?terms=Arthur%20G.%20West
Arthur G. West New York 1930 2
http://www.newspapers.com/image/560010078?terms=Arthur%20G.%20

Dixie Dixon Ohio 1936 2
http://www.newspapers.com/image/392508247?terms=Dixie%20Dixon
http://www.newspapers.com/image/400805747?terms=Dixie%20Dixon
Dixie Dixon Pennsylvania 1935 5
http://www.newspapers.com/image/524448629?terms=Dixie%20Dixon
http://www.newspapers.com/image/140511332?terms=Dixie%20Dixon
http://www.newspapers.com/image/140512402?terms=Dixie%20Dixon
http://www.newspapers.com/image/182214268?terms=Dixie%20Dixon
Dolly Levins Michigan 1936 2
http://www.newspapers.com/image/206648099?terms=Dolly%20Levins
Dolly O'Day Michigan 1937 2
http://www.newspapers.com/image/34328930?terms=Dolly%20O'Day
Don Holly Maryland 1935 1
Don Kenyard California 1937 1
file not found: /Volumes/GoogleDrive/My Drive/performer-searches-by-year/don_kenyard/n1940-california.json
Don Rand Kentucky 1937 1
file not found: /Volumes/GoogleDrive/My Drive/performer-searches-by-year/don_rand/n1940-michigan.json
Don Rand New York 1935 1
http://www.newspapers.com/image/686049187?terms=Don%20Rand
Don Rand Texas 19

KeyError: 'Montreal, Canada'

In [10]:
df['Year'] = df.apply(lambda row: pd.to_datetime(row.Date).year, axis=1)
df['Unnamed'] = df.apply(lambda row: 'unnamed' in row.Performer.lower(), axis=1)
filtered_df = df[(df["Unnamed"]==False) & (df["Year"] >= 1930) & (df["Year"] <= 1939) & (df["State"] != '')]

In [11]:
by_performer = dict()
by_state = dict()
for (performer, state, year), rows in filtered_df.groupby(['Performer', 'State', 'Year']):
    by_performer.setdefault(performer, {})
    by_performer[performer].setdefault(state, {})
    by_performer[performer][state].setdefault(year, [])
    
    by_state.setdefault(state, {})
    by_state[state].setdefault(performer, {})
    by_state[state][performer].setdefault(year, [])
    
    newspaper_ids = set(str(int(x)) for x in rows.Newspaper_ID if x)
    eima_ids = set(str(int(x)) for x in rows.EIMA_ID if x)
    ids = list(newspaper_ids | eima_ids)
    
    if ids:
        by_performer[performer][state][year] = ids
        by_state[state][performer][year] = ids

3